In [1]:
import geemap
import ee
import os
from Monarch import user_gee
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [5]:
os.chdir(r'E:\Data\Remote sensing')

In [6]:
shp = geemap.shp_to_ee("JXt.shp")
Map.addLayer(shp, {}, "shp")

ProxyError: ('Cannot connect to proxy.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
roi = shp.geometry()
s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
        .filterDate('2021-09', '2021-10')\
        .filterBounds(roi)\
        .sort('CLOUDY_PIXEL_PERCENTAGE', False)
Map.addLayer(s2.mosaic().clip(roi), {'bands':['B4', 'B3', 'B2'], 'max':3000, 'min': 0}, 's2')

In [ ]:
s2.aggregate_array('system:id').getInfo()